In [ ]:
# This code is building a Conv-LSTM model to predict the next frame sample from MOVING MNIST dataset.
# Importing libraries:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os

In [ ]:
# Downloading dataset:
saving_dir = os.path.join(os.getcwd(), "Moving_MNIST")
if not os.path.isdir(saving_dir):
    os.mkdir(saving_dir)
dataset_dir = os.path.join(saving_dir, "moving_mnist.npy")
if not os.path.isdir(dataset_dir):
    keras.utils.get_file(
    dataset_dir,
    "http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy",)

In [ ]:
# setting some parameters:
epochs = 20
batch_size = 3

In [ ]:
# Loading the dataset
mvmnist_data = np.load(dataset_dir)
# The loaded dataset has shape of sequence, samples, height, width, therefore we need to swap first two axes.
mvmnist_data = np.swapaxes(mvmnist_data, 0, 1)

In [ ]:
# Normalizing and splitting the dataset into train and validation parts with train_ratio for train part.
# data_ratio is the amount of the whole data that we want to use
def preprocessing(dataset_dir, train_ratio = 0.8, data_ratio = 0.2, seq_length = 19):
    # Loading the dataset:
    mvmnist_data = np.load(dataset_dir)
    # The loaded dataset has shape of sequence, samples, height, width, therefore we need to swap first two axes.
    mvmnist_data = np.swapaxes(mvmnist_data, 0, 1)
    mvmnist_data = mvmnist_data[:int(data_ratio*mvmnist_data.shape[0])]
    # Normalizing data
    mvmnist_data = mvmnist_data/ 255
    # Expand the image dimension by one to have (image_height, image_width, 1) shape.
    mvmnist_data = np.expand_dims(mvmnist_data, axis = -1)
    
    # Creating train and validation sets.
    indices = np.arange(mvmnist_data.shape[0])
    np.random.shuffle(indices)
    train_set = mvmnist_data[:int(train_ratio*mvmnist_data.shape[0])]
    validation_set = mvmnist_data[-int(train_ratio*mvmnist_data.shape[0]):]
    
    # now we are shifting the train sequence to have the next frames for target.
    # Checking if the input seq_length is wrongly imported to be bigger than the existing sequence length in dataset.
    if seq_length >= train_set.shape[1]:
        seq_length = train_set.shape[1] - 1
    
    train_shifted = train_set[:, 0:seq_length-1, :, :]
    train_shifted_target = train_set[:, 1:seq_length, :, :]
    validation_shifted = train_set[:, 0:seq_length-1, :, :]
    validation_shifted_target = validation_set[:, 1:seq_length, :, :]
    return train_shifted, train_shifted_target, validation_shifted, validation_shifted_target    

# Creating Conv-LSTM model

In [ ]:
# Conv_LSTM function gets two inputs:
# n_number: number of neurons for each Conv-LSTM layer (number of elements will be the same as CL_number) ex: n_number = [32, 64]
# it means there will be two Conv-LSTM layers with 32 and 64 neurons.
# sequence_size: sequence, wdith, and height of the image. ex (20, 480, 640, 1)
def Conv_LSTM(n_number, sequence_size):
    input_var = layers.Input(shape = sequence_size)
    x = layers.ConvLSTM2D(n_number[0],
                         5,
                         padding = "same",
                         return_sequences = True,
                         activation = "relu")(input_var)
    x = layers.BatchNormalization()(x)
    for layer_n in range(len(n_number)-1):
        x = layers.ConvLSTM2D(n_number[layer_n + 1],
                              5,
                              padding = "same",
                              return_sequences = True,
                              activation = "relu")(x)
        x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(1, kernel_size=3, activation="sigmoid", padding="same")(x)
    
    model = keras.models.Model(input_var, x)
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam())
    return model


In [ ]:
# Creating dataset
train_shifted, train_shifted_target, validation_shifted, validation_shifted_target  = preprocessing(dataset_dir, train_ratio = 0.8, data_ratio = 0.1, seq_length = 19)

In [ ]:
my_model = Conv_LSTM([128, 128, 128], train_shifted.shape[1:]) 
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)


# Fit the model to the training data.
my_model.fit(
    train_shifted,
    train_shifted_target,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_shifted, validation_shifted_target),
    callbacks=early_stopping)